# Overview

In this tutorial we will introduce both Google Colab and the Tensorflow 2.0 / Keras API, including demonstration of basic concepts related to statistical modeling and machine learning. An overview of topics covered in this tutorial include:

**Google Colab**

* Jupyter notebooks
* mounting Google drive
* environment setup

**Tensorflow 2.0 API**

* Tensorflow graphs
* creating models
* creating optimizers
* creating loss functions
* model fitting

This tutorial is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

# Google Colab

The following lines of code will configure your Google Colab environment for this tutorial. For those interested in running a local Jupyter server, please consider using one of the following options to ensure dependency compatibility:

1. Precompiled Docker images: see https://github.com/peterchang77/install (**recommended**)
2. Conda environment files: see https://github.com/peterchang77/dl_utils/tree/master/envs

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [ ]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Throughout this tutorial we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [ ]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This tutorial will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [ ]:
# --- Select Tensorflow 2.0 (only in Google Colab)
%tensorflow_version 2.x

### Jupyter

A Jupyter notebook is composed of blocks of `Markdown` documentation or code referenced as cells. Each cell can be individually selected by a simple click. As you progress through this notebook, simply select a code-containing cell and click the `Run` button on the top toolbar (or alternatively `shift` + `[Enter]`) to execute that particular line or block of code. The `In [ ]` header to the left of each cell will change status to `In [*]` while a line or block of code is executing and then to a number indicating a line or block of executed code if successful.

# Tensorflow 2.0 and Keras

Tensorflow is a free and open-source software library developed by the Google Brain team for dataflow and differentiable programming
across a range of tasks. It is a symbolic math library, and is most popularly used for machine learning applications such as neural networks. In November 2019, the first stable release of the verson 2.0 library was made available, with significant changes including:

* formal integration of the high-level Keras API for easy model building
* `eager execution` of code, eliminating the need to manually compile man abstract syntax tree using a `session.run()` call
* improved support for model deployment in production on any platform
* improved support for distributed machine learning paradigms

More information highlighting the key improvements can be found here: https://www.tensorflow.org/guide/effective_tf2

## Import

In this tutorial we will use the following Numpy and Tensorflow library components:

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import numpy as np
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers

## Machine learning 

Machine learning models are **mapping functions** that learn to predict target output(s) based on provided input(s). Inputs may consist of a number of **features** derived from raw data (traditional machine learning models) or simply the **raw data** itself (neural networks). 

A machine learning model is defined by its **parameters**, numeric variables that are applied via **operations** on the algorithm inputs to yield desired outputs. Note that this defintion of learning is broad; in fact all conventional statistical models are systems with *learnable* parameters.  

Consider a simple linear regression model:

```
y = m * x + b
```

As per above, this model meets all key specifications of a learnable system:

* input: `x`
* output: `y`
* parameters: `m` and `b`

Indeed, through repeated exposure to data samples, optimal values for `m` and `b` can be learned such that the target output `y` can be reliably predicted from any given input `x`.

## Tensorflow graphs

To implement a model with Tensorflow, one must reformulate an algorithm into a computational graph: a series of **operations** that define use of **parameters** to map provided input(s) to target output(s).

In the above simple linear regression model, the multiplication (`m`) and addition (`b`) operators are combined into a single operation known as a **linear transformation**. In the context of neural networks, this operation is also synonmous with the term **densely-connected layer**, a concept that will be covered in future tutorials. 

In [ ]:
# --- Define an input
x = Input(shape=(1,))

# --- Define a linear transform operation
op = layers.Dense(1)

# --- Apply linear transform
y = op(x)

In Tensorflow, only input(s), output(s), intermediate values and operations are explicitly defined. By contrast parameters are maintained (and updated) implicitly by the Tensorflow library upon definition of operations. In other words, `m` and `b` in our model are created automatically by Tensorflow as soon as the operation (`op`) is defined. By default in Tensorflow, multiplication parameters are set to random values (chosen via the `Glorot` intialization scheme) whereas addition parameters are set to zero. 

In [ ]:
# --- See parameters
m, b = op.get_weights()
print(m)
print(b)

## Creating Models

Once the input(s), output(s) and all required operations have been defined, a Tensorflow `Model()` object can be defined:

In [ ]:
# --- Define model by passing input(s) and output(s)
model = Model(inputs=x, outputs=y)

To pass an arbitrary value (in the form of a NumPy array) into the model, use the `model.predict(...)` function:

In [ ]:
# --- Pass an input into the model
model.predict(np.array([1]))

For complex models, it may be useful to visualize a summary of all intermediate operations:

In [ ]:
# --- Print summary of model architecture
model.summary()

## Compiling a Model

The current model parameters have been initialized to random values. Through exposure to data, the goal is for the model to *learn* optimal parameter values that allow for robust mapping of provided input to target output. To prepare the model for learning, a graph must be **compiled** through definition of, at minimum, the key following training components (each represented by Keras Python objects):

* loss function
* optimizer

### Defining a loss object

A loss function simply represents a formula that the machine can use to provide feedback regarding the quality of its current set of parameters. In other words, given a provided input `x` and a target output `y`, as well as model prediction `y'`, how does one quantify the *goodness* of the estimated output? Choosing a representative loss function is important as this feedback is used by the machine to improve its parameter values.

In machine learning, any loss formulation can be used to estimate goodness of fit as long as the function is **differentiable**. Many pre-built loss functions encapsulated by Python classes are availabe for use in the `tf.losses.*` module.

For a linear regression model, performance (e.g. fit) is most commonly evaluated by calculating the *squared distance* between the target output `y` and the model prediction `y'`. In other words, if a model predicts `5` when the target output is `2`, then the error is `(5 - 2) ** 2` or `9`. Thus, the parameters `m` and `b` that yield the **least squared error** for all data observations is defined to be optimal. 

In [ ]:
# --- Define a MSE loss
loss = losses.MeanSquaredError()

### Defining an optimizer object

An optimizer is a method used by the machine to improve its parameters. By definition, the parameters are updated such that the loss value (calculated by the loss function) decreases. A number of optimization methods have been described and are available through the `tf.optimizers.*` module. Currently, one of the most effective optimizers is the Adam technique which will be used in this tutorial (a good default choice for most tasks). 

In addition to optimizer technique, a learning rate specifying the *degree of change* per update step is required. For the purposes of this tutorial, we will use a default learning rate of `2e-4`.

In [ ]:
# --- Define an Adam optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

### Compiling

Once the model `optimizer` and `loss` objects have been defined, simply pass these objects into the `model.compile(...)` method to prepare for training:

In [ ]:
# --- Compile model
model.compile(
    optimizer=optimizer,
    loss=loss)

The model is now compiled and ready for training!

# Data

For demonstration of algorithm training, data "samples" will be simulated using the following code:

In [ ]:
# --- Define arbitrary linear function constants
m = 2
b = -1

# --- Define lambda function
data = lambda x : m * x + b + np.random.rand(*x.shape)

Here the lambda function `data` essentially maps the input `x` via a linear transformation with some additional random noise. Use the following lines of code to visualize this data:

In [ ]:
# --- Generate N number examples of data
N = 100
xs = np.random.rand(N)
ys = data(xs)

# --- Visualize
import pylab
pylab.scatter(xs, ys)

### Python generators

# Model Training

## Saving and Loading a Model

After a model has been successfully trained, it can be saved and/or loaded by simply using the `model.save()` and `models.load_model()` methods. Note that any custom losses and/or metrics will need to be provided via a dictionary.

In [ ]:
# --- Serialize a model
model.save('./intro.hdf5')

In [ ]:
# --- Load a serialized model
del model
model = models.load_model('./intro.hdf5')

In [ ]:
# --- Delete saved model
import os
os.remove('./intro.hdf5')